In [ ]:
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path,PurePath


































































































































 


Path.cwd()
path = Path(r'D:\Users\GPatil13\Downloads')
os.chdir(path)
os.getcwd()
os.listdir()

 

main_df = pd.read_excel('Area_Sutures_out - Copy.xlsx')
df =main_df.sample(frac =0.16)
df.shape

 

df.head()

 

use_cols = ['Description','USP', 'Length', 'Suture Color', 'Needle Name','Brandname']
df = df[use_cols]
df

 

df['USP'] = df['USP'].str.replace('-0','')

 

df['USP'].value_counts()#.count()

 

df['Length'].value_counts().count(), df['Brandname'].value_counts().count()

 

# for col in df.iloc[:,1:].columns:
#     print(df[col].value_counts())

 

def preprocess(text):
    text = text.lower() #lowercase text
    text=text.strip() #get rid of leading/trailing whitespace
    text=re.compile('|<.*?>').sub('', text) #Remove HTML tags/markups
    text = re.sub('\s+', ' ', text) #Remove extra space and tabs
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\s+',' ',text) #\s matches any whitespace, \s+ matches multiple whitespace, \S matches non-whitespace

 

    return text

 

df['Description'] = df['Description'].apply(lambda x: preprocess(x))

 

df.head()#['Description']#df_usp =  

 

### Check Nan values

 

df.isnull().sum()

 

df = df.dropna()

 

### lets use tf-idf now

 

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# for bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

 

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

 

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

 

### Lets build the model for USP TV

 

df1[ df1['Description']=='vicryl']#.count()

 

#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split( df['Description'], df['USP'],test_size=0.3,shuffle=True, random_state=1)
X_train
# As Word2vec runs on Tocckenized data so need to convert into tockens
# X_train_tok = [nltk.word_tokenize(i) for i in X_train]
# X_test_tok =  [nltk.word_tokenize(i) for i in X_test]

 

#Tf-Idf
tfidf_vectorizer = TfidfVectorizer( ngram_range=(1,2),use_idf=True)

 

X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
X_train_vectors_tfidf

 

#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2',random_state=1)
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  

 

## Lets predict y value for the test dataset now
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

 

y_predict[:12], y_prob[:12]

 

f1_score = f1_score(y_test, y_predict, average='weighted')
f1_score

 

print(classification_report(y_test, y_predict))

 

print('Confusion Matrix:\n',confusion_matrix(y_test, y_predict))

 

#### Since ROC-AUC score is onlyapplicable for Binary class classification hence wecant use it here. 

 

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

### ## Lets Try SVM

 

## Lets Try SVM
from sklearn import svm   #, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

 

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1,random_state=1).fit(X_train_vectors_tfidf, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=2,random_state=1).fit(X_train_vectors_tfidf, y_train)

 

To calculate the efficiency of the two models, we’ll test the two classifiers using the test data set:

 

## Lets predict y value for the test dataset now
poly_pred = poly.predict(X_test_vectors_tfidf)
#poly_proba = poly.predict_proba(X_test_vectors_tfidf)[:,1]

 

rbf_pred = rbf.predict(X_test_vectors_tfidf)
#rbf_proba = rbf.predict_proba(X_test_vectors_tfidf)[:,1]

 

Finally, we’ll calculate the accuracy and f1 scores for SVM with Polynomial kernel:

 

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

 

print(classification_report(y_test, poly_pred))

 

In the same way, the accuracy and f1 scores for SVM with RBF kernel:

 

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

 

### Now lets use another TV = 'Length'

 

df.head()

 


#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test =train_test_split(df['Description'],df['Length'],test_size=0.3,shuffle=True,random_state=1)

 


X_train

 

#Tf-Idf
tfidf_vectorizer = TfidfVectorizer( ngram_range=(1,2),use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
X_train_vectors_tfidf

 

#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2',random_state=1)
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  

 

## Lets predict y value for the test dataset now
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

 

y_predict[:12], y_prob[:12]

 

print(accuracy_score(y_test, y_predict))
print(f1_score(y_test, y_predict, average='weighted'))

 

### ## Lets Try SVM

 

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1,random_state=1).fit(X_train_vectors_tfidf, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1,random_state=1).fit(X_train_vectors_tfidf, y_train)

 

To calculate the efficiency of the two models, we’ll test the two classifiers using the test data set:

 

## Lets predict y value for the test dataset now
poly_pred = poly.predict(X_test_vectors_tfidf)
#poly_proba = poly.predict_proba(X_test_vectors_tfidf)[:,1]

 

rbf_pred = rbf.predict(X_test_vectors_tfidf)
#rbf_proba = rbf.predict_proba(X_test_vectors_tfidf)[:,1]

 

Finally, we’ll calculate the accuracy and f1 scores for SVM with Polynomial kernel:

 

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

 

print(classification_report(y_test, poly_pred))

 

In the same way, the accuracy and f1 scores for SVM with RBF kernel:

 

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')

 

print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

 

### Now lets build models on 3rd TV = 'Suture Color'

 

 df['Suture Color'].value_counts()

 

from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
df['Suture Color']= label_encoder.fit_transform(df['Suture Color'])
 
df['Suture Color'].unique()

 

#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df['Description'],df['Suture Color'],
                                                    test_size=0.3,shuffle=True,random_state=1)

 

X_train

 

#Tf-Idf
tfidf_vectorizer = TfidfVectorizer( ngram_range=(1,2),use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
X_train_vectors_tfidf

#### Try Logistic Regression

 

#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2',random_state=1)
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  

 

## Lets predict y value for the test dataset now
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

 

y_predict[:12], y_prob[:12]

 

print(accuracy_score(y_test, y_predict))
print(f1_score(y_test, y_predict, average='weighted'))

 

#### Try SVM

 

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1,random_state=1).fit(X_train_vectors_tfidf, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1,random_state=1).fit(X_train_vectors_tfidf, y_train)

 

To calculate the efficiency of the two models, we’ll test the two classifiers using the test data set:

 

## Lets predict y value for the test dataset now
poly_pred = poly.predict(X_test_vectors_tfidf)
#poly_proba = poly.predict_proba(X_test_vectors_tfidf)[:,1]

 

rbf_pred = rbf.predict(X_test_vectors_tfidf)
#rbf_proba = rbf.predict_proba(X_test_vectors_tfidf)[:,1]

 

 

Finally, we’ll calculate the accuracy and f1 scores for SVM with Polynomial kernel:

 

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

 

print(classification_report(y_test, poly_pred))

 

In the same way, the accuracy and f1 scores for SVM with RBF kernel:

 

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')

 

print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

 

### Lets try now Last TV = 'Brandname'

 

df.head()

 

from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
df['Brandname']= label_encoder.fit_transform(df['Brandname'])
 
df['Brandname'].unique()

 

#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split( df['Description'],df['Brandname'],test_size=0.3,shuffle=True,random_state=1)

 


X_train

 

#Tf-Idf
tfidf_vectorizer = TfidfVectorizer( ngram_range=(1,2),use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
X_train_vectors_tfidf

 

#### Try Logistic Regression

 

#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2',random_state=1)
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  

 

## Lets predict y value for the test dataset now
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

 

y_predict[:12], y_prob[:12]

 

print(accuracy_score(y_test, y_predict))
print(f1_score(y_test, y_predict, average='weighted'))

 

#### Try SVM

 

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1,random_state=1).fit(X_train_vectors_tfidf, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1,random_state=1).fit(X_train_vectors_tfidf, y_train)

 

To calculate the efficiency of the two models, we’ll test the two classifiers using the test data set:

 

## Lets predict y value for the test dataset now
poly_pred = poly.predict(X_test_vectors_tfidf)
#poly_proba = poly.predict_proba(X_test_vectors_tfidf)[:,1]

 

rbf_pred = rbf.predict(X_test_vectors_tfidf)
#rbf_proba = rbf.predict_proba(X_test_vectors_tfidf)[:,1]

 

Finally, we’ll calculate the accuracy and f1 scores for SVM with Polynomial kernel:

 

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

 

print(classification_report(y_test, poly_pred))

 

In the same way, the accuracy and f1 scores for SVM with RBF kernel:

 

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')

 

print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))